# Import Required Libraries

In [1]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00
time: 1.51 ms (started: 2023-01-28 02:06:35 +00:00)


In [2]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from pandas import Grouper
from sklearn.impute import KNNImputer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import datetime
import io
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"
import requests
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

time: 11.1 s (started: 2023-01-28 02:06:35 +00:00)


# Dataset Preparation

In [3]:
#Downloading the cleaned csv file from our GitHub account
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
download = requests.get(url).content

#Reading the downloaded content and turning it into a pandas dataframe
covid_data = pd.read_csv(io.StringIO(download.decode('utf-8')))

time: 10.3 s (started: 2023-01-28 02:06:46 +00:00)


In [4]:
#We will select a single country to analyze for our Time Series Forecasting Models. 
#In this case, we have selected the United States.
covid_usa_data = covid_data[covid_data["location"].isin(['United States'])]

time: 41 ms (started: 2023-01-28 02:06:57 +00:00)


In [5]:
#Convert the date column from string to datetime in covid_usa_data.
covid_usa_data['date'] = pd.to_datetime(covid_usa_data['date'])

time: 17.8 ms (started: 2023-01-28 02:06:57 +00:00)


In [6]:
#Delete columns where over 50% of the values are null as these would be difficult to impute.
covid_usa_data.dropna(thresh = len(covid_usa_data) * .5, axis = 1, inplace = True)

time: 25.6 ms (started: 2023-01-28 02:06:57 +00:00)


In [7]:
#Remove columns with only 1 distinct value. 
#These columns will not have any predictive value to the target variable.
covid_attributes_1 = [c for c in list(covid_usa_data) if len(covid_usa_data[c].unique()) > 1]

time: 26.3 ms (started: 2023-01-28 02:06:57 +00:00)


In [8]:
#Only include metrics that are updated daily, such as the number of new cases, deaths, hospitalizations, tests, vaccinations, and ICU patients, that occur on a daily basis. 
#We also want to include raw counts only and not values that have been averaged or smoothed over time as the data will be grouped by week later on.
covid_attributes_2 = ['date', 'new_cases', 'new_deaths', 'reproduction_rate', 
                      'icu_patients', 'hosp_patients', 'new_tests', 
                      'tests_per_case', 'new_vaccinations', 'stringency_index']
covid_usa_data = covid_usa_data[covid_attributes_2]

time: 11 ms (started: 2023-01-28 02:06:57 +00:00)


In [9]:
#Filter data from February 1st onwards to delete null values for the 'new_cases' column.
covid_usa_data_filtered = covid_usa_data[covid_usa_data['date'] >= '02-01-2020']

time: 4.25 ms (started: 2023-01-28 02:06:57 +00:00)


In [10]:
#Attain the percentage of missing values per column to understand how clean our dataset is.
percent_missing = covid_usa_data_filtered.isnull().sum() * 100 / len(covid_usa_data_filtered)
percent_missing = percent_missing.sort_values(ascending=True)
print(percent_missing[percent_missing > 0])
print(len(percent_missing[percent_missing > 0]))

stringency_index      2.383135
new_deaths            2.749771
reproduction_rate     5.224565
icu_patients         15.215399
hosp_patients        15.215399
new_tests            23.006416
tests_per_case       23.556370
new_vaccinations     29.880843
dtype: float64
8
time: 17.1 ms (started: 2023-01-28 02:06:57 +00:00)


In [11]:
#Get the percentage of missing values per column. 
percent_missing = covid_usa_data_filtered.isnull().sum() * 100 / len(covid_usa_data_filtered)
percent_missing = percent_missing.sort_values(ascending=True)
print(percent_missing[percent_missing > 0])
print(len(percent_missing[percent_missing > 0]))

stringency_index      2.383135
new_deaths            2.749771
reproduction_rate     5.224565
icu_patients         15.215399
hosp_patients        15.215399
new_tests            23.006416
tests_per_case       23.556370
new_vaccinations     29.880843
dtype: float64
8
time: 20.4 ms (started: 2023-01-28 02:06:57 +00:00)


In [12]:
#If imputing column using KNNImputer, initialize neighbors value to be used for future imputations
imputer = KNNImputer(n_neighbors=5)

time: 659 µs (started: 2023-01-28 02:06:57 +00:00)


In [13]:
#The first reported COVID-19 death in the USA was 02/19/2020. Set all 'new_death' values before 02/19/2020 to 0. 
non_null_date_d = covid_usa_data_filtered['date'].loc[covid_usa_data_filtered['new_deaths'].first_valid_index()]
non_null_date_d_str = non_null_date_d.strftime('%Y-%m-%d')
covid_usa_data_filtered['new_deaths'] = covid_usa_data_filtered.apply(lambda x: 0 if 
                x['date'].strftime('%Y-%m-%d') < non_null_date_d_str else x['new_deaths'], axis=1)

time: 66.8 ms (started: 2023-01-28 02:06:57 +00:00)


In [14]:
#Find all non-null values for 'new_vaccinations' before the first non-null value and inpute to 0. This represents time period where the vaccine did not exist. 
non_null_date_v = covid_usa_data_filtered['date'].loc[covid_usa_data_filtered['new_vaccinations'].first_valid_index()]
non_null_date_v_str = non_null_date_v.strftime('%Y-%m-%d')
covid_usa_data_filtered['new_vaccinations'] = covid_usa_data_filtered.apply(lambda x: 0 if 
                x['date'].strftime('%Y-%m-%d') < non_null_date_v_str else x['new_vaccinations'], axis=1)

time: 71.7 ms (started: 2023-01-28 02:06:57 +00:00)


In [15]:
#Percentage of missing values per column
percent_missing_2 = covid_usa_data_filtered.isnull().sum() * 100 / len(covid_usa_data_filtered)
percent_missing_2 = percent_missing_2.sort_values(ascending=True)
print(percent_missing_2[percent_missing_2 > 0])
print(len(percent_missing_2[percent_missing_2 > 0]))

new_deaths            0.183318
new_vaccinations      0.824931
stringency_index      2.383135
reproduction_rate     5.224565
icu_patients         15.215399
hosp_patients        15.215399
new_tests            23.006416
tests_per_case       23.556370
dtype: float64
8
time: 13.8 ms (started: 2023-01-28 02:06:57 +00:00)


In [16]:
#Only select columns where the percentage of null values is less than 10% as these would be easy to impute.
covid_usa_data_filtered = covid_usa_data_filtered[['date','new_cases','new_deaths',
                                                             'new_vaccinations','stringency_index',
                                                             'reproduction_rate']]
covid_usa_data_filtered.head()

,date,new_cases,new_deaths,new_vaccinations,stringency_index,reproduction_rate
237814,2020-02-01,0.0,0.0,0.0,0.00,NaN
237815,2020-02-02,0.0,0.0,0.0,5.56,NaN
237816,2020-02-03,3.0,0.0,0.0,5.56,NaN
237817,2020-02-04,0.0,0.0,0.0,5.56,NaN
237818,2020-02-05,0.0,0.0,0.0,5.56,NaN


time: 70.1 ms (started: 2023-01-28 02:06:57 +00:00)


In [17]:
#Use the KNNImputer class to effectively replace null values in the following columns: 'new_deaths', 'new_vaccinations', 'stringency_index', and 'reproduction_rate'. 
covid_usa_data_filtered['new_deaths'] = imputer.fit_transform(covid_usa_data_filtered[['new_deaths']])
covid_usa_data_filtered['new_vaccinations'] = imputer.fit_transform(covid_usa_data_filtered[['new_vaccinations']])
covid_usa_data_filtered['stringency_index'] = imputer.fit_transform(covid_usa_data_filtered[['stringency_index']])
covid_usa_data_filtered['reproduction_rate'] = imputer.fit_transform(covid_usa_data_filtered[['reproduction_rate']])

time: 71 ms (started: 2023-01-28 02:06:57 +00:00)


In [18]:
#Ensure that there are no negative values in dataframe. 
print ((covid_usa_data_filtered[['new_cases', 'new_deaths', 'new_vaccinations', 
                      'stringency_index', 'reproduction_rate']] < 0).any().any())

False
time: 10.9 ms (started: 2023-01-28 02:06:57 +00:00)


In [19]:
#Use the pd.Grouper function to get data on weekly basis.
covid_usa_data_filtered = covid_usa_data_filtered.groupby([pd.Grouper(key='date', freq='W')]).mean().reset_index()
covid_usa_data_filtered = covid_usa_data_filtered.set_index('date')
covid_usa_data_filtered.head()

,new_cases,new_deaths,new_vaccinations,stringency_index,reproduction_rate
date,,,,,
2020-02-02,0.000000,0.000000,0.0,2.780000,1.080938
2020-02-09,0.571429,0.000000,0.0,5.560000,1.080938
2020-02-16,0.285714,0.000000,0.0,5.560000,1.080938
2020-02-23,0.285714,0.000000,0.0,5.560000,1.080938
2020-03-01,2.285714,0.142857,0.0,5.955714,1.080938


time: 90.8 ms (started: 2023-01-28 02:06:57 +00:00)


# Model Development

In [20]:
#Create the features and target variable
target_col = "reproduction_rate"
features = list(covid_usa_data_filtered.columns.difference([target_col]))

#We want to forecast new vaccinations by 3 months into the future, which is 13 weeks. 
forecast_lead = 13
target = f"{target_col}_lead{forecast_lead}"

covid_usa_data_filtered[target] = covid_usa_data_filtered[target_col].shift(-forecast_lead)
covid_usa_data_filtered = covid_usa_data_filtered.iloc[:-forecast_lead]

time: 18.7 ms (started: 2023-01-28 02:06:57 +00:00)


In [21]:
covid_usa_data_filtered.reset_index(inplace=True)

time: 3.33 ms (started: 2023-01-28 02:06:57 +00:00)


In [22]:
#Get train and test data
split = int(round((len(covid_usa_data_filtered) * 0.75), 0))
cutoff_date = covid_usa_data_filtered['date'].astype(str).iloc[split]
cutoff_date_next = covid_usa_data_filtered['date'].astype(str).iloc[split+1]
covid_usa_data_filtered = covid_usa_data_filtered.set_index('date')
covid_train = covid_usa_data_filtered.loc[:cutoff_date].copy()
covid_test = covid_usa_data_filtered.loc[cutoff_date_next:].copy()

time: 16.7 ms (started: 2023-01-28 02:06:58 +00:00)


In [23]:
#Standardize the features and target
target_mean = covid_train[target].mean()
target_stdev = covid_train[target].std()

for c in covid_train.columns:
    mean = covid_train[c].mean()
    stdev = covid_train[c].std()
    covid_train[c] = (covid_train[c] - mean) / stdev
    covid_test[c] = (covid_test[c] - mean) / stdev

time: 12.1 ms (started: 2023-01-28 02:06:58 +00:00)


In [24]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

time: 3.2 ms (started: 2023-01-28 02:06:58 +00:00)


In [25]:
torch.manual_seed(101)

batch_size = 32
sequence_length = 30

train_dataset = SequenceDataset(
    covid_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    covid_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([32, 30, 4])
Target shape: torch.Size([32])
time: 122 ms (started: 2023-01-28 02:06:58 +00:00)


In [26]:
from torch import nn

class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

time: 2.96 ms (started: 2023-01-28 02:06:58 +00:00)


In [27]:
learning_rate = 0.05
num_hidden_units = 15
model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

time: 8.5 ms (started: 2023-01-28 02:06:58 +00:00)


# Model Training

In [28]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")


print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(10):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 0.12246114481240511

Epoch 0
---------
Train loss: 0.9301932454109192
Test loss: 0.14358896389603615

Epoch 1
---------
Train loss: 0.9359118044376373
Test loss: 0.1648225486278534

Epoch 2
---------
Train loss: 0.7306423783302307
Test loss: 0.1447249106131494

Epoch 3
---------
Train loss: 0.4971144050359726
Test loss: 0.15511980932205915

Epoch 4
---------
Train loss: 0.33077552542090416
Test loss: 0.161899721249938

Epoch 5
---------
Train loss: 0.28267678432166576
Test loss: 0.20153556764125824

Epoch 6
---------
Train loss: 0.1848597154021263
Test loss: 0.19347690790891647

Epoch 7
---------
Train loss: 0.22952108271420002
Test loss: 0.24429146945476532

Epoch 8
---------
Train loss: 0.21288684383034706
Test loss: 0.2193986400961876

Epoch 9
---------
Train loss: 0.24734123423695564
Test loss: 0.47100791335105896

time: 1.14 s (started: 2023-01-28 02:06:58 +00:00)


# Model Evaluation

In [29]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output


train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
covid_train[ystar_col] = predict(train_eval_loader, model).numpy()
covid_test[ystar_col] = predict(test_loader, model).numpy()

df_out = pd.concat((covid_train, covid_test))[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

print(df_out)

            reproduction_rate_lead13  Model forecast
date                                                
2020-02-02                  0.965714        0.890665
2020-02-09                  0.920000        0.896745
2020-02-16                  0.927143        0.895767
2020-02-23                  0.947143        0.895784
2020-03-01                  0.958571        0.896705
...                              ...             ...
2022-10-02                  0.924286        0.918960
2022-10-09                  1.052233        0.916419
2022-10-16                  1.080938        0.914644
2022-10-23                  1.080938        0.916298
2022-10-30                  1.080938        0.915403

[144 rows x 2 columns]
time: 42.1 ms (started: 2023-01-28 02:06:59 +00:00)


In [30]:
rmse = (mean_squared_error(df_out['reproduction_rate_lead13'], df_out['Model forecast'])) ** 0.5
r2 = r2_score(df_out['reproduction_rate_lead13'], df_out['Model forecast'])
print ("RMSE: {}".format(rmse))
print ("R^2: {}".format(r2))

RMSE: 0.09271200492461161
R^2: 0.7510711733604932
time: 3.58 ms (started: 2023-01-28 02:06:59 +00:00)


In [31]:
plot_template = dict(layout=go.Layout({"font_size": 18,
                           "xaxis_title_font_size": 24,
                           "yaxis_title_font_size": 24}))

fig = px.line(df_out, labels=dict(created_at="Date", value="Reproduction Rate"))
fig.add_vline(x=cutoff_date, line_width=4, line_dash="dash")
fig.add_annotation(xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False)
fig.update_layout(
    template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()

time: 562 ms (started: 2023-01-28 02:06:59 +00:00)
